In [1]:
from BasisExpansion import BasisExpansion
import numpy as np
from LinClass import LinClass
from scipy.special import expit

In [2]:
heartData = np.loadtxt('cleanedSAheart.data.txt', skiprows=1)[:, 1:]
arrayY = heartData[:,-1]
y = np.zeros((len(heartData), 2))
for ii in range(len(heartData)):
    if arrayY[ii] == 0:
        y[ii] = np.array([1, 0])
    elif arrayY[ii] == 1:
        y[ii] = np.array([0, 1])
    else:
        print('something wrong')
        print(ii)
        print(y[ii])
X = heartData[:,0:-1]

In [3]:
heartBaseExpanded = BasisExpansion()
XTrain, XTest, yTrain, yTest = X[:int(len(X)/3)], X[int(len(X)/3):], y[:int(len(y)/3)], y[int(len(y)/3):]
expandedXTrain = np.zeros((len(XTrain), 4*(len(XTrain[0]) - 1) + 1))
expandedXTest = np.zeros((len(XTest), 4*(len(XTest[0])- 1) + 1))

In [4]:
index = 0
for ii in range(len(XTrain[0])):
    if ii == 4:
        expandedXTrain[:,index] = XTrain[:,ii]
        expandedXTest[:,index] = XTest[:, ii]
        index +=1
    else:
        col = XTrain[:,ii]
        knots = heartBaseExpanded.getKnots(5, col)
        for knot in knots:
            expandedXTrain[:,index] = heartBaseExpanded.piecewiseLinear(XTrain[:,ii], knot)
            expandedXTest[:,index] = heartBaseExpanded.piecewiseLinear(XTest[:,ii], knot)
            index +=1

In [5]:
heartClass = LinClass(expandedXTrain, yTrain, indicatorArray = ['no heart attack', 'heart attack'])

In [6]:
print(expandedXTrain)

[[57.   31.25  5.5  ... 24.75 12.5   0.25]
 [41.   15.25  0.   ... 35.75 23.5  11.25]
 [15.    0.    0.   ... 18.75  6.5   0.  ]
 ...
 [25.    0.    0.   ... 24.75 12.5   0.25]
 [19.    0.    0.   ...  0.    0.    0.  ]
 [ 5.    0.    0.   ...  0.    0.    0.  ]]


In [7]:
heartClass.binaryLogisticRegg()
print(heartClass.logisticReggBestFit)

[-0.85079967 -0.52289381  0.28819442  0.86887413 -0.78052854  0.77016844
 -0.07329828 -6.57956893  7.19585376  0.30749264  0.48609296 -0.04998285
 -0.50814122 -0.18908388  0.87202512 -0.15646891  0.1122885   0.11601256
  7.94639927 -8.14033016  0.52297531  0.66105114 -1.75159324  1.121144
 -0.70551844  1.03705887 -0.41382277  0.54834612  0.49470486 -0.83053537
  3.74890437 -3.36797708  0.2632431   0.24873721]


In [8]:
print(expandedXTest.shape)
print(expandedXTrain.shape)

(308, 33)
(154, 33)


In [9]:
XtestStandardized = heartClass.standardizeTest(expandedXTest)
logisticFit = heartClass.logisticReggBestFit
print(logisticFit)
correct = 0
total = 0
for ii in range(len(XtestStandardized)):
    x = XtestStandardized[ii]
    probability = expit(-np.dot(x, logisticFit))
    if probability > 1/2:
        result = np.array([1, 0])
    else:
        result = np.array([0, 1])
    if (result == yTest[ii]).all():
        correct+=1 
    total +=1
print(correct/total)

[-0.85079967 -0.52289381  0.28819442  0.86887413 -0.78052854  0.77016844
 -0.07329828 -6.57956893  7.19585376  0.30749264  0.48609296 -0.04998285
 -0.50814122 -0.18908388  0.87202512 -0.15646891  0.1122885   0.11601256
  7.94639927 -8.14033016  0.52297531  0.66105114 -1.75159324  1.121144
 -0.70551844  1.03705887 -0.41382277  0.54834612  0.49470486 -0.83053537
  3.74890437 -3.36797708  0.2632431   0.24873721]
0.6883116883116883
